# OD Estimation Test
## Comparing Estimation Algorithms
#### Evan Faulkner
#### 3/29/2021

### Notation and algorithms described in:
#### IPF: https://dspace.mit.edu/handle/1721.1/37970
#### CS: https://arxiv.org/pdf/1404.3263.pdf
####     https://journals.sagepub.com/doi/pdf/10.1177/0361198119845896

In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import cvxpy as cp

### Generate Single Line OD Problem

We generate a random OD matrix for a one-way bus route and calculate the boarding and alighting counts $B_i$ and $A_j$

A seed matrix needs to be initialized for the IPF algorithm. The seed can either be a diffuse/uniform prior which seems to work in the absence of a better seed. Alternatively, we can use a seed matrix which is more representative of the true OD matrix by adding zero mean noise to the entries or subtracting some proportion of the true counts.

In [2]:
N_stops = 5  # number of stops on the bus line
i_max = 20  # max flow from i to j
X_true = np.triu(np.random.randint(0,i_max+1,size=(N_stops,N_stops)),1)  # true OD matrix, upper triangular, zero main diagonal
B = np.sum(X_true,1)  # boarding counts at stop i
A = np.sum(X_true,0)  # alighting counts at stop j
mask = X_true > 0  # mask zero elements
X_seed = (np.sum(B)/((N_stops**2 - N_stops)/2)) * np.triu(np.ones((N_stops,N_stops)),1)  # diffuse seed matrix
#t_ij = np.maximum(T_ij - np.triu(np.random.randint(0,math.ceil(0.7*i_max),\
#                    size=(N_stops,N_stops)),1),np.ones((N_stops,N_stops)))*mask  # seed matrix with unreliable counts

### IPF

The IPF algorithm attempts to find the OD matrix closest to the seed matrix which satisfies the marginal sum constraints given by the boarding and alighting counts. Thus, the prior/seed is very important to the quality of the estimate.

We calculate "row factors" $a_i$ and "column factors" $b_j$ such that
$$\sum_{j=1}^{N} b_i x_{ij} = B_i, \qquad \sum_{i=1}^{N} a_j x_{ij} = A_j$$
where $x_{ij}$ is the flow from stop $i$ to stop $j$, the $ij$ element of the OD matrix $X$, $B_i$ is the boarding count at stop $i$, and $A_j$ is the alighting count at stop $j$.
The IPF algorithm works by calculating row and column factors iteratively, applying the previously calculated row/column factor to solve for the next column/row factor respectively. When the factor updates converge, we calculate the entries of the IPF estimated OD matrix as the product of the row and column factors with the corresponding entry of the seed matrix.
$$\left\{\widehat{X}_{IPF}\right\}_{ij} = a_ib_j\left\{X_{seed}\right\}_{ij}$$

In [3]:
eps = 1e-3  # Stop when delta is less than eps
delta = 1  # delta is the difference between the last 2 iterations
a_i = B/np.sum(X_seed,1)  # a_i is the row factor, initialized with b_j^0=1
a_i[np.isnan(a_i)] = 0
a_i[a_i<1] = 1
a_i = np.expand_dims(a_i,1)
b_j = A/np.sum(a_i*X_seed,0)  # b_j is the column factor, initialized with a_i calculated above
b_j[np.isnan(b_j)] = 0
b_j[b_j<1] = 1
b_j = np.expand_dims(b_j,1)

X_ipf = np.zeros((N_stops,N_stops))

while True:
    if delta > eps:
        a = B/np.sum(b_j[:,-1].T*X_seed,1)
        a[np.isnan(a)] = 0
        a[a<1] = 1
        a = np.expand_dims(a,1)
        b = A/np.sum(a*X_seed,0)
        b[np.isnan(b)] = 0
        b[b<1] = 1
        b = np.expand_dims(b,1)
        delta = np.abs(np.mean(a-a_i[:,-1])+np.mean(b-b_j[:,-1]))
#         print('delta = ',delta)
        a_i = np.c_[a_i,a]
        b_j = np.c_[b_j,b]
    else:
        print(f'a_i = {np.round(a_i[:,-1], 2)}', f'\nb_j = {np.round(b_j[:,-1], 2)}')
        break
    
for i in range(N_stops):
    for j in range(N_stops):
        X_ipf[i,j] = a_i[i,-1]*b[j,-1]*X_seed[i,j]  # Compute the estimated OD matrix from the final a_i, b_j, and seed entries

a_i = [1.   1.24 1.41 1.04 1.  ] 
b_j = [1.   1.   1.12 1.   1.  ]


<ipython-input-3-f137715d58a1>:3: RuntimeWarning: invalid value encountered in true_divide
  a_i = B/np.sum(X_seed,1)  # a_i is the row factor, initialized with b_j^0=1
<ipython-input-3-f137715d58a1>:7: RuntimeWarning: invalid value encountered in true_divide
  b_j = A/np.sum(a_i*X_seed,0)  # b_j is the column factor, initialized with a_i calculated above
<ipython-input-3-f137715d58a1>:16: RuntimeWarning: invalid value encountered in true_divide
  a = B/np.sum(b_j[:,-1].T*X_seed,1)
<ipython-input-3-f137715d58a1>:20: RuntimeWarning: invalid value encountered in true_divide
  b = A/np.sum(a*X_seed,0)


### Compressed Sensing

Since we know for the single line OD estimation problem that the matrix is upper triangular, we can use compressed sensing techniques to recover a sparse OD matrix estimate that satisfies the constraints. We accomplish this by minimizing the $L_1$ norm of the vectorized OD matrix subject to the marginal constraints given by the boarding and alighting counts, as well as the structural constraints on the matrix. These constraints can be represented as a linear system of equations
$$ A\text{vec}(X) = b$$
The same constraints can also be written individually.
$$\sum_{j=1}^N x_{ij} = b_i$$
$$\sum_{i=1}^N x_{ij} = a_j$$
$$x_{ij} \geq 0$$
$$x_{ij} = 0 \text{ if } i\geq j$$

In [4]:
X = cp.Variable((N_stops,N_stops))
obj = cp.Minimize(cp.norm(cp.vec(X),1))
constraints = [
    cp.sum(X,axis=0,keepdims=False) == A,  # OD estimate must match alighting counts
    cp.sum(X,axis=1,keepdims=False) == B,  # OD estimate must match boarding counts
    cp.upper_tri(cp.transpose(X)) == 0,  # No passengers are time travelers
    cp.diag(X) == 0,  # No passenger gets on and off at the same stop
    X >= 0  # Passenger counts are nonnegative
]
prob = cp.Problem(obj,constraints)
prob.solve()
X_cs = X.value
print("status:", prob.status)

status: optimal


### Maximum Entropy

We can use the same constraints with different objectives to obtain different OD estimates. Here, we find the maximum entropy solution subject to the same constraints above.

In [5]:
X = cp.Variable((N_stops,N_stops))
obj = cp.Maximize(cp.sum(cp.entr(X)))
constraints = [
    cp.sum(X,axis=0,keepdims=False) == A,  # OD estimate must match alighting counts
    cp.sum(X,axis=1,keepdims=False) == B,  # OD estimate must match boarding counts
    cp.upper_tri(cp.transpose(X)) == 0,  # No passengers are time travelers
    cp.diag(X) == 0,  # No passenger gets on and off at the same stop
    X >= 0  # Passenger counts are nonnegative
]
prob = cp.Problem(obj,constraints)
prob.solve()
X_me = X.value
print("status:", prob.status)

status: optimal


### Minimum 2-norm

This is the same as above with the objective as the minimum $L_2$ norm of the vectorized matrix

In [6]:
X = cp.Variable((N_stops,N_stops))
obj = cp.Minimize(cp.norm(cp.vec(X),2))
constraints = [
    cp.sum(X,axis=0,keepdims=False) == A,  # OD estimate must match alighting counts
    cp.sum(X,axis=1,keepdims=False) == B,  # OD estimate must match boarding counts
    cp.upper_tri(cp.transpose(X)) == 0,  # No passengers are time travelers
    cp.diag(X) == 0,  # No passenger gets on and off at the same stop
    X >= 0  # Passenger counts are nonnegative
]
prob = cp.Problem(obj,constraints)
prob.solve()
X_l2 = X.value
print("status:", prob.status)

status: optimal


### Minimum infinity-norm

This is the same as above with the objective as the minimum $L_\infty$ norm of the vectorized matrix

In [7]:
X = cp.Variable((N_stops,N_stops))
obj = cp.Minimize(cp.norm(cp.vec(X),"inf"))
constraints = [
    cp.sum(X,axis=0,keepdims=False) == A,  # OD estimate must match alighting counts
    cp.sum(X,axis=1,keepdims=False) == B,  # OD estimate must match boarding counts
    cp.upper_tri(cp.transpose(X)) == 0,  # No passengers are time travelers
    cp.diag(X) == 0,  # No passenger gets on and off at the same stop
    X >= 0  # Passenger counts are nonnegative
]
prob = cp.Problem(obj,constraints)
prob.solve()
X_inf = X.value
print("status:", prob.status)

status: optimal


### Comparison

In [8]:
print(f'Error in X_ipf: {np.round(np.sum(np.abs(X_true-X_ipf))/np.sum(np.abs(X_true)),2)}')

Error in X_ipf: 0.42


In [9]:
print(f'Error in X_cs: {np.round(np.sum(np.abs(X_true-X_cs))/np.sum(np.abs(X_true)),2)}')

Error in X_cs: 0.26


In [10]:
print(f'Error in X_me: {np.round(np.sum(np.abs(X_true-X_me))/np.sum(np.abs(X_true)),2)}')

Error in X_me: 0.27


In [11]:
print(f'Error in X_l2: {np.round(np.sum(np.abs(X_true-X_l2))/np.sum(np.abs(X_true)),2)}')

Error in X_l2: 0.31


In [12]:
print(f'Error in X_inf: {np.round(np.sum(np.abs(X_true-X_inf))/np.sum(np.abs(X_true)),2)}')

Error in X_inf: 0.4


In [13]:
X_true

array([[ 0,  5,  4,  5,  8],
       [ 0,  0, 20, 12,  5],
       [ 0,  0,  0,  9, 18],
       [ 0,  0,  0,  0, 10],
       [ 0,  0,  0,  0,  0]])

In [14]:
X_ipf.round()

array([[ 0., 10., 11., 10., 10.],
       [ 0.,  0., 13., 12., 12.],
       [ 0.,  0.,  0., 14., 14.],
       [ 0.,  0.,  0.,  0., 10.],
       [ 0.,  0.,  0.,  0.,  0.]])

In [15]:
X_cs.round()

array([[ 0.,  5.,  7.,  5.,  5.],
       [ 0.,  0., 17.,  9., 11.],
       [ 0.,  0.,  0., 12., 15.],
       [ 0.,  0.,  0.,  0., 10.],
       [-0.,  0.,  0.,  0.,  0.]])

In [16]:
X_me.round()

array([[ 0.,  5.,  8.,  4.,  5.],
       [ 0.,  0., 16.,  9., 11.],
       [ 0.,  0.,  0., 12., 15.],
       [ 0.,  0.,  0.,  0., 10.],
       [-0.,  0.,  0.,  0.,  0.]])

In [17]:
X_l2.round()

array([[ 0.,  5.,  9.,  3.,  5.],
       [ 0.,  0., 15., 10., 12.],
       [ 0.,  0.,  0., 13., 14.],
       [ 0.,  0.,  0.,  0., 10.],
       [-0.,  0.,  0.,  0.,  0.]])

In [18]:
X_inf.round()

array([[ 0.,  5., 11.,  2.,  5.],
       [ 0.,  0., 13., 11., 13.],
       [ 0.,  0.,  0., 13., 14.],
       [ 0.,  0.,  0.,  0., 10.],
       [-0.,  0.,  0.,  0.,  0.]])